<a href="https://colab.research.google.com/github/joegonzalezMSL/Warm-Dense-Matter-2025/blob/main/lammps-minimal/examples/phase_transition/Cu-nucleation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get -q install lammps

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  fonts-mathjax lammps-data lammps-doc lammps-examples libfftw3-double3
  libjs-mathjax libkim-api2 liblammps0 libvoro++1
Suggested packages:
  openkim-models libfftw3-bin libfftw3-dev fonts-mathjax-extras fonts-stix
  libjs-mathjax-doc
The following NEW packages will be installed:
  fonts-mathjax lammps lammps-data lammps-doc lammps-examples libfftw3-double3
  libjs-mathjax libkim-api2 liblammps0 libvoro++1
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 66.2 MB of archives.
After this operation, 278 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-mathjax all 2.7.9+dfsg-1 [2,208 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 lammps-data all 20220106.git7586adbb6a+ds1-2 [9,729 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 lammps-exampl

In [2]:

# --- CONFIG ---
REPO_URL = "https://github.com/joegonzalezMSL/Warm-Dense-Matter-2025.git"
BRANCH   = "main"
# --- CLONE ---
!rm -rf /content/*
!git clone -b "$BRANCH" "$REPO_URL" /content/repo

# --- GO INTO REPO ---
%cd /content/repo/lammps-minimal/examples/phase_transition/

Cloning into '/content/repo'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 188 (delta 68), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (188/188), 605.65 KiB | 3.86 MiB/s, done.
Resolving deltas: 100% (68/68), done.
/content/repo/lammps-minimal/examples/phase_transition


In [3]:
!pwd

/content/repo/lammps-minimal/examples/phase_transition


In [4]:
!lmp -h | head -n2 |tail -1

Large-scale Atomic/Molecular Massively Parallel Simulator - 29 Sep 2021 - Update 2


# Crystallization of A Supercooled Liquid Copper

We study crystallization in a supercooled copper liquid, focusing on homogeneous nucleation as the escape from metastability and showing how cooling rate dictates supercooling depth and the liquid–solid transition.

I. Stable and Metastable Phases

A stable phase is a state of matter (solid, liquid, or gas) where the system has minimized its free energy at a given temperature and pressure. At equilibrium, the system has no net tendency to change its phase.
Sometimes, a system can exist in a state that is not the most stable but persists for some time. This is called a metastable phase. For example, liquid water can exist below its normal freezing point without crystallizing—this is known as supercooling.
Thus, supercooling occurs when a liquid is cooled below its equilibrium freezing temperature but does not immediately crystallize. The farther the system is from equilibrium, the more unstable the metastable state becomes. Eventually, the system undergoes a phase transition to the more stable phase—here, the solid.
The lifetime of the metastable supercooled state decreases as the system moves further below the equilibrium temperature, eventually leading to nucleation of the solid phase.

II. Nucleation Fundamentals

When a liquid is cooled below its equilibrium freezing point, crystallization does not begin instantly. Instead, the transition to the solid phase starts with the formation of small crystalline clusters within the liquid. This process is called nucleation.
At first, atoms in the liquid may come together and form tiny crystalline embryos. However, most of these are unstable and dissolve back into the liquid. Only when a cluster reaches a certain size — called the critical nucleus size — does it become stable enough to keep growing. Below this size, surface effects dominate and the cluster is more likely to disappear; above this size, the reduction in energy from forming a solid outweighs the cost of creating new surfaces, and the nucleus grows spontaneously.
Nucleation reflects a balance between two competing free-energy contributions: the bulk term, which stabilizes the solid and drives growth, and the surface term, which penalizes the creation of a new interface. The balance between these two determines the critical size.
The nucleation rate — how quickly new solid clusters appear — depends strongly on temperature, or in other words, on supercoolings. Just below the melting point, the driving force for crystallization is small, so nucleation is very slow. As the liquid is cooled further, the driving force increases and nucleation becomes more likely. Important: if the system is cooled too quickly, atoms may not have enough time to rearrange into crystalline order, and the liquid can bypass nucleation, remaining in a metastable supercooled state or becoming an amorphous solid (glass).
Two types of nucleation can occur: (1) Homogeneous nucleation: Nuclei form randomly in the bulk liquid; (2) Heterogeneous nucleation: Nuclei form preferentially at surfaces, impurities, or defects, which lower the energy barrier and make nucleation easier. In practice, heterogeneous nucleation is far more common in real systems.
Once a stable nucleus forms, the process enters the growth stage, where atoms from the surrounding liquid attach to the solid cluster, and the crystal expands until the system transforms to the stable phase.

III. Molecular Dynamics Simulation

We use a reliable model for copper to explore its supercooled state and observe homogeneous, spontaneous crystal nucleation. The procedure is simple: we first heat a crystalline sample well above its melting temperature to ensure complete melting, and then cool it back down.
In the simulation, the main indicators of nucleation are: (1) A sudden drop in potential energy — this happens because atoms in the liquid adopt the more energetically favorable crystalline arrangement; (2) The emergence of ordered structures — initially small, but once stable nuclei form, they grow into a crystalline phase.

In [26]:
lammps_input = """
# ----------------------------------
# LAMMPS script: Cu Supercooling
# ----------------------------------

# 1. Initialize simulation
units metal
dimension 3
boundary p p p
atom_style atomic

# 2. Create FCC crystal
lattice fcc 3.615
region box block 0 6 0 6 0 6
create_box 1 box
create_atoms 1 box

# 3. MD model
pair_style eam/alloy
pair_coeff 		* * /content/Warm-Dense-Matter-2025/lammps-minimal/potential/Cu_zhou.eam.alloy Cu
neighbor 2.0 bin
neigh_modify delay 10
timestep 0.002 # = 2 fs

# 4. Output
thermo 1000
thermo_style custom step temp press pe etotal
dump myDump all atom 1000 dump_cu.txt

# 5. Equilibrate at high T to melt
velocity all create 3000 12345 mom yes rot yes
fix melt all npt temp 2000 2000 0.1 iso 0 0 1.0
run 10000
unfix melt


# -----------------------------
# 6. Cooling
# -----------------------------
variable Tstart equal 2000      # Start temperature (K)
variable Tend equal 300         # End temperature (K)

fix cool all npt temp ${Tstart} ${Tend} 0.1 iso 0 0 1.0

run 200000  # adjust to control final T and cooling rate

unfix cool

# -----------------------------
# 7. Save final configuration
# -----------------------------
write_data cu_final.data

# -----------------------------
# 8. Data analysis
# -----------------------------
# After the run, you can plot:
# 1. Temperature vs. Step: column 2 in thermo output
# 2. Pressure vs. Step: column 3
# 3. Potential energy vs. Step: column 4
# Example: using gnuplot:
# gnuplot> plot 'log.lammps' using 1:2 with lines title 'Temperature'
# -----------------------------
"""
with open("npt.in",'w') as f:
  f.write(lammps_input)



In [27]:
!lmp -in npt.in

LAMMPS (29 Sep 2021 - Update 2)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 3.6150000 3.6150000 3.6150000
Created orthogonal box = (0.0000000 0.0000000 0.0000000) to (21.690000 21.690000 21.690000)
  1 by 1 by 1 MPI processor grid
Created 864 atoms
  using lattice units in orthogonal box = (0.0000000 0.0000000 0.0000000) to (21.690000 21.690000 21.690000)
  create_atoms CPU = 0.000 seconds
Reading eam/alloy potential file /content/Warm-Dense-Matter-2025/lammps-minimal/potential/Cu_zhou.eam.alloy with DATE: 2007-10-12
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 11.0375
  ghost atom cutoff = 11.0375
  binsize = 5.51875, bins = 4 4 4
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair eam/alloy, perpetual
      attributes: half, newton on
      pair build: half/bin/a

## 4. Homework
Modify the cooling rate in the LAMMPS input file:

1.   Slow down the cooling by factors of 10.
2.   Speed it up by a factor of 10 and 100.

For each case, analyze the potential energy as a function of time. Compare how the cooling rate influences the onset of crystallization, the degree of supercooling reached, and the sharpness of the energy drop. Summarize your findings and explain how the rate of cooling controls the balance between remaining in the metastable liquid state and transitioning to the crystalline solid.